In [1]:
import re
import shutil
import pandas as pd
from pandas import DataFrame, Series
import github3
from github3 import login
from git import Repo, GitCommandError
import yaml
from subprocess import Popen, PIPE, STDOUT, SubprocessError, check_output
from configparser import ConfigParser
from github3.repos.branch import Branch
from github3.null import NullObject
from schematics.models import Model, BaseType
from schematics.types import StringType, IntType, BooleanType
from schematics.types.compound import ListType, DictType
from schematics.exceptions import ValidationError
%cd ~/Documents/projects/stitch/python/
from stitch.core.stitch_frame import StitchFrame
%cd ~/Documents/projects/nerve/python/
from nerve.core.project_manager import ProjectManager
from nerve.core.project import Project
from nerve.core.git_remote import GitRemote
from nerve.core.utils import *
from nerve.spec.base import *
from nerve.core.git import Git
from nerve.core.git_lfs import GitLFS
from nerve.core.metadata import Metadata
from nerve.core.utils import execute_subprocess, get_status
# from nerve.core.api import *
from nerve.spec.specifications import *
from nerve.spec.validators import *
from nerve.spec.traits import *
from nerve.spec import traits
from nerve.test.test_suite import *
%cd /tmp/ntr002

/Users/alex/Documents/projects/stitch/python
/Users/alex/Documents/projects/nerve/python
/private/tmp/ntr002


In [2]:
conf = '/Users/alex/Documents/projects/nerve/python/nerve/resources/nerverc.yml'
man = ProjectManager(conf)

In [6]:
man.delete('ntr002', False, True)

True

In [3]:
man.config['timeout'] = 30
man.clone('ntr002')
man.config['timeout'] = 100

In [ ]:
man.create('ntr002', 'some note about this project')

In [4]:
%%bash
rm -rf /tmp/ntr002-new
cp -R /tmp/ntr002/ /tmp/ntr002-old
cd /tmp/ntr002/
mkdir vol001/ntr002_vol001_desc_v001
echo v001 sdjfkh 0000 > vol001/ntr002_vol001_desc_v001/ntr002_vol001_desc_v001_0000.png
echo v001 jyelkh 0001 > vol001/ntr002_vol001_desc_v001/ntr002_vol001_desc_v001_0001.png
echo v001 lwekfj 0002 > vol001/ntr002_vol001_desc_v001/ntr002_vol001_desc_v001_0002.png
echo vasc v001 > maya001/ntr002_maya001_vasc_v001.mb
echo kidney v001 > maya001/ntr002_maya001_kdny_v001.mb
tree

.
├── geo001
├── maya001
│   ├── ntr002_maya001_kdny_v001.mb
│   └── ntr002_maya001_vasc_v001.mb
├── ntr002_proj001_meta.yml
└── vol001
    └── ntr002_vol001_desc_v001
        ├── ntr002_vol001_desc_v001_0000.png
        ├── ntr002_vol001_desc_v001_0001.png
        └── ntr002_vol001_desc_v001_0002.png

4 directories, 6 files


cp: /tmp/ntr002-old/.git/objects/pack/pack-19f6d02b168cd837d703b921befc63e7d4f061d9.idx: Permission denied
cp: /tmp/ntr002-old/.git/objects/pack/pack-19f6d02b168cd837d703b921befc63e7d4f061d9.pack: Permission denied


In [3]:
stat = list(man.status('ntr002'))
data = None
if len(stat) > 0:
    data = DataFrame(list(stat)).applymap(lambda x: x.data)
    data = StitchFrame(data).flatten(prefix=False).to_dataframe().T
data

,0
asset-id,adce540f-cad4-43bd-ac1d-ab...
asset-name,ntr002_vol001_desc_v001
asset-type,deliverable
data,None
dependencies,[]
descriptor,desc
notes,
project-id,74616146
project-name,ntr002
project-url,git@github.com:nerve-test-...


In [4]:
man.publish('ntr002', 'some note from the alex', verbosity=2)

True

In [5]:
%%bash
mv /tmp/ntr002 /tmp/ntr002-new
mv /tmp/ntr002-old/ /tmp/ntr002

In [7]:
man.request('ntr002')

True